In [52]:
import time
import cProfile
import math
from multiprocess import Pool
import random
from mpi4py import MPI

In [58]:
# parameters
n = 1_000_000
num_processes = 4

## Without Parallelization

In [54]:
from python import compute_pi
pr = cProfile.run('compute_pi(n)')

         2000127 function calls (2000126 primitive calls) in 0.465 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.430    0.430 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 decorator.py:199(fix)
        1    0.000    0.000    0.027    0.027 decorator.py:229(fun)
        1    0.000    0.000    0.027    0.027 history.py:54(only_when_enabled)
        1    0.005    0.005    0.011    0.011 history.py:824(_writeout_input_cache)
        1    0.000    0.000    0.000    0.000 history.py:830(_writeout_output_cache)
        1    0.000    0.000    0.027    0.027 history.py:836(writeout_cache)
        4    0.000    0.000    0.000    0.000 inspect.py:2782(name)
       10    0.000    0.000    0.000    0.000 inspect.py:2794(kind)
        1    0.000    0.000    0.000    0.000 inspect.py:2874(__init__)
        1    0.000    0.000    0.000    0.000 inspect.py:2882(args)
        1    0.000

In [55]:
def f(x):
    return math.sqrt(1 - x**2)

def compute_pi(n):
    dx = 1.0 / n
    sum = 0
    for i in range(n):
        x = i * dx
        sum += f(x) * dx
    return 4 * sum

n = 10_000_000

start_time = time.time()
approx_pi = compute_pi(n)
end_time = time.time()

print(f"Execution time: {end_time - start_time} sec" )
print(f"Approximation of pi: {approx_pi}")

Execution time: 1.7551953792572021 sec
Approximation of pi: 3.1415928535523587


## With Parallel Computing via Multiprocessing

In [56]:
def compute_pi(n, num_processes):
    chunk_size = n // num_processes
    pool = Pool(processes=num_processes)
    results = []
    for _ in range(num_processes):
        result = pool.apply_async(monte_carlo_pi, args=(chunk_size,))
        results.append(result)
    pool.close()
    pool.join()

    total_in_circle = sum(result.get() for restult in results)
    return 4 * total_in_circle / n

def monte_carlo_pi(chunk_size):
    count_in_circle = 0
    for _ in range(chunk_size):
        x = random.random()
        y = random.random()
        if x**2 + y**2 <= 1:
            count_in_circle += 1
    return count_in_circle

n = 1_000_000
num_processes = 4

start_time = time.time()
approx_pi = compute_pi(n, num_processes)
end_time = time.time()

print(f"Execution time: {end_time - start_time} sec" )
print(f"Approximation of pi: {approx_pi}")

Execution time: 0.14556312561035156 sec
Approximation of pi: 3.138704


## With Distributed Parallel Computing via mi4py

In [57]:
def f(x):
    return math.sqrt(1 - x**2)

def compute_partial_sum(start, end, n):
    dx = 1.0 / n
    partial_sum = 0
    for i in range(start, end):
        x = i * dx
        partial_sum += f(x) * dx
    return partial_sum

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

n = 100_000  

chunk_size = n // size
remainder = n % size
if remainder != 0:
    if rank < remainder:
        chunk_size += 1

start = rank * chunk_size
end = start + chunk_size

start_time = time.time()
partial_sum = compute_partial_sum(start, end, n)
total_sum = comm.reduce(partial_sum, op=MPI.SUM, root=0)

if rank == 0:
    approx_pi = 4 * total_sum
    end_time = time.time()
    print(f"Execution time: {end_time - start_time} sec" )
    print(f"Approximation of pi: {approx_pi}")

comm.barrier()

Execution time: 0.01860523223876953 sec
Approximation of pi: 3.1416126164019564
